# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

**Criação** de arquivo csv para extração

In [2]:
# na verdade não recebi a planilha então gerei a minha com base do que vem no endpoint de users da api

import requests
import csv
import json

response = requests.get(f'{sdw2023_api_url}/users')

data = json.loads(response.content)
headers = data[0].keys()

# só printei os cabeçalhos para saber se veio alguma coisa, eu poderia ter validado de alguma forma com um "if headers" mas fica para um outro momento
print(headers)

with open('SDW2023.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    writer.writerows(data)

dict_keys(['id', 'name', 'account', 'card', 'features', 'news'])


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [11]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')

# neste ponto só ajustei o user_id para id pois no print dos cabeçalhos não apresentou user_id
user_ids = df['id'].tolist()
print(user_ids)

[9, 116, 1, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 132, 136, 137, 138, 139, 8, 5, 6, 10, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 252, 189, 190, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 23

In [5]:
import requests
import json

def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 9,
    "name": "Ayumi",
    "account": {
      "id": 12,
      "number": "00003-03",
      "agency": "0001",
      "balance": 0.0,
      "limit": 1000.0
    },
    "card": {
      "id": 9,
      "number": "**** **** **** 0003",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 130,
        "icon": "\ud83e\udd11",
        "description": null
      },
      {
        "id": 138,
        "icon": "\ud83e\udd11",
        "description": null
      },
      {
        "id": 146,
        "icon": "\ud83e\udd11",
        "description": "Ayumi, N\u00e3o espere. Invista agora e garanta seu amanh\u00e3."
      },
      {
        "id": 154,
        "icon": "\ud83e\udd11",
        "description": "Ayumi, Investir \u00e9 cuidar do seu futuro financeiro."
      },
      {
        "id": 240,
        "icon": "https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg",
        "description": "Eduardo esteve aqui"
      },
      {
        "id

## **T**ransform


~~Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.~~

Na falta de tokens para uso do openai, resolvi utilizar uma biblioteca bem legal para diferenciar este projeto.
Manterei o código original para evidenciar as diferenças na solução.

In [6]:
# !pip install openai

In [7]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
# openai_api_key = 'TODO'

**Download** do script necessário para a solução.
- todos os créditos deste script são dele: https://github.com/MatheusMuriel/LeroLeroPy

In [8]:
# aqui eu resolvi baixar o script visto que não há a necessidade de manter no repositório

lerolero_response = requests.get('https://raw.githubusercontent.com/MatheusMuriel/LeroLeroPy/master/lerolero.py', allow_redirects=True)
open('lerolero.py', 'wb').write(lerolero_response.content)

7731

In [9]:
# import openai

# openai.api_key = openai_api_key

# def generate_ai_news(user):
#   completion = openai.ChatCompletion.create(
#    model="gpt-4",
#    messages=[
#      {
#           "role": "system",
#           "content": "Você é um especialista em markting bancário."
#       },
#       {
#           "role": "user",
#           "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
#       }
#     ]
#   )
#   return completion.choices[0].message.content.strip('\"')

import lerolero

for user in users:
  lero = f'{user.get("name")}, {lerolero.geraFrase().lower()}'
  print(lero)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": lero
  })

Ayumi, percebemos, cada vez mais, que o aumento do diálogo entre os diferentes setores produtivos prepara-nos para enfrentar situações atípicas decorrentes do fluxo de informações. 
Maria, ainda assim, existem dúvidas a respeito de como o aumento do diálogo entre os diferentes setores produtivos agrega valor ao estabelecimento das direções preferenciais no sentido do progresso. 
Devweekerson, no entanto, não podemos esquecer que o comprometimento entre as equipes oferece uma interessante oportunidade para verificação das novas proposições. 
anguile, não obstante, o comprometimento entre as equipes prepara-nos para enfrentar situações atípicas decorrentes do investimento em reciclagem técnica. 
Ronaldo, todas estas questões, devidamente ponderadas, levantam dúvidas sobre se o entendimento das metas propostas não pode mais se dissociar dos níveis de motivação departamental. 
anguile, ainda assim, existem dúvidas a respeito de como o desafiador cenário globalizado talvez venha a ressaltar

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [10]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")

{'id': 9, 'name': 'Ayumi', 'account': {'id': 12, 'number': '00003-03', 'agency': '0001', 'balance': 0.0, 'limit': 1000.0}, 'card': {'id': 9, 'number': '**** **** **** 0003', 'limit': 1000.0}, 'features': [], 'news': [{'id': 130, 'icon': '🤑', 'description': None}, {'id': 138, 'icon': '🤑', 'description': None}, {'id': 146, 'icon': '🤑', 'description': 'Ayumi, Não espere. Invista agora e garanta seu amanhã.'}, {'id': 154, 'icon': '🤑', 'description': 'Ayumi, Investir é cuidar do seu futuro financeiro.'}, {'id': 240, 'icon': 'https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg', 'description': 'Eduardo esteve aqui'}, {'id': 241, 'icon': 'https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg', 'description': 'Eduardo esteve aqui'}, {'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg', 'description': 'Ayumi, percebemos, cada vez mais, que o aumento do diálogo entre os diferentes setores produtivos prepara-nos